# やること
- テレビcmによる効果を推定する
- 年齢や性別という共変量を持つ場合を考える

<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>

In [1]:
import random
import numpy as np
np.random.seed(1234)
random.seed(1234)

import pandas as pd
from scipy.special import expit
from numpy.random import *
import scipy.stats
import os

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [2]:
num_data = 200

# 年齢変数（15から76の一様乱数）
x_1 = randint(15, 76, num_data)

# 性別（1を男性）
x_2 = randint(0, 2, num_data)

# ノイズ
e_z = randn(num_data)

# シグモイド間数
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうか
Z = np.array([])

# シグモイド関数からテレビを見たか確率計算して作る
for i in range(num_data):
    Z_i = np.random.choice(2, size = 1, p = [1 - z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

In [3]:
# 購入量Yを定義
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [4]:
# データフレームで見る
df = pd.DataFrame({'age': x_1,
                   'sex': x_2,
                   'watch_cm': Z,
                   'purchase_amount': Y,
                   })

df.head()  # 先頭を表示

,age,sex,watch_cm,purchase_amount
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [5]:
# 平均値で比較してみる
print(df.groupby('watch_cm').mean())

                age       sex  purchase_amount
watch_cm                                      
0.0       32.141026  0.692308        68.827143
1.0       55.836066  0.483607        49.711478


cmを見るかどうかのシグモイド関数は女性であり、高齢であるほど値が大きくなる。一方で、購入量は若く男性の方が大きくなるので、cmの効果が正しく推定できていない。

In [6]:
# 回帰分析を実行
from sklearn.linear_model import LinearRegression

x = df[['age', 'sex', 'watch_cm']]
y = df['purchase_amount']

model = LinearRegression().fit(x, y)

print(model.coef_)

[-0.95817951 32.70149412 10.41327647]


# 傾向スコアを用いたIPW

調整化公式を用いた変形を行うと以下の通りとなる。


$$P(Y = y|do(Z=z))=\sum_{x}{P(x, y, z)/P(z|x)}$$

In [7]:
from sklearn.linear_model import LogisticRegression

# 傾向スコアの推定。ロジスティック回帰を使う。
x = df[['age', 'sex']]
z = df['watch_cm']

reg = LogisticRegression().fit(x, z)

print('係数', reg.coef_)
print('切片', reg.intercept_)

係数 [[ 0.10562765 -1.38263933]]
切片 [-3.37146523]


In [8]:
z_pred = reg.predict_proba(x)
print(z_pred[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


平均処置効果は
$$ATE = E(Y_1) - E(Y_0) = E(Y_1|do(Z=1)) - E(Y_0|do(Z=0))$$

調整化公式を用いて変形すると

$$ATE = \sum_{x}E(Y|Z=1, X=x)P(X=x) - \sum_{x}E(Y|Z=0, X=x)P(X=x)$$

$$= \frac{1}{N}\sum_{i}^{N} \frac{y_{i}}{P(Z=1|X=x_{i})}Z_{i} - \frac{1}{N}\sum_{i}^{N} \frac{y_{i}}{P(Z=0|X=x_{i})}(1-Z_{i})$$

In [9]:
ate_i = y/z_pred[:, 1]*z - y/z_pred[:, 0]*(1-z)
ate = 1/len(y)*ate_i.sum()
print('ATE', ate)

ATE 8.847476810855458


# Doubly Robust Estimation

In [10]:
# 回帰分析によるyの推定
# ===================================
x = df[['age', 'sex', 'watch_cm']]
y = df['purchase_amount']

model = LinearRegression().fit(x, y)

# z=0の場合の予測値作る
x_0 = x.copy()
x_0['watch_cm'] = 0
y_0 = model.predict(x_0)

# z=1の場合の予測値作る
x_1 = x.copy()
x_1['watch_cm'] = 1
y_1 = model.predict(x_1)

# ロジスティック回帰によるzの推定
# ===================================
x = df[['age', 'sex']]
z = df['watch_cm']

reg = LogisticRegression().fit(x, z)

z_pred = reg.predict_proba(x)

In [11]:
ate_1_i = y/z_pred[:, 1]*z + (1 - z/z_pred[:, 1])*y_1
ate_0_i = y/z_pred[:, 0]*(1-z) + (1-(1 - z)/z_pred[:, 0])*y_0
ate = 1/len(y)*(ate_1_i - ate_0_i).sum()
print('ATE', ate)

ATE 9.75277505424846
